# Charts for mojort paper

In [ ]:
import pandas as pd
import seaborn as sns
from benchkit.charts.printedcharts import export_figure
import matplotlib.pyplot as plt
import benchkit
#import cairosvg
from benchkit.charts.dataframes import get_dataframe
import os
from pathlib import Path
from benchkit.utils.dir import gitmainrootdir
import re

In [ ]:
repo_dir = gitmainrootdir()
# paper_dir = Path("/home/antonio/Dropbox/Applications/ShareLaTeX/rtns25_mojort")
paper_dir = Path("/tmp")

fig_dir = paper_dir / "figures/generated"

os.makedirs(str(fig_dir), exist_ok=True)

In [ ]:
def make_parent(
    path: Path,
) -> None:
    parent_dir = Path(path).parent.resolve()
    parent_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
def export_fig(
    plot,
    path: Path,
) -> None:
    make_parent(path)

    export_figure(
        plot=plot,
        path=path,
        creation_year=2024,
        creation_month=12,
        creation_day=10,
    )

In [ ]:
CONTEXT = "talk"
STYLE = "whitegrid"
PALETTE = "colorblind"
# ESTIMATOR = np.median
ESTIMATOR_STR = "median"
# CONFINT = 'sd'
FIG_USE_LATEX_FONTS = True

In [ ]:
def set_config(width=8, height=6, font_scale=1.15):
    sns.set_theme(
        context=CONTEXT,
        style=STYLE,
        palette=PALETTE,
        font_scale=font_scale,
        rc={
            "figure.figsize": (width, height),
            "pdf.fonttype": 42,
            "pdf.use14corefonts": True,
            "text.usetex": FIG_USE_LATEX_FONTS,
        },
    )

In [ ]:
set_config()

In [ ]:
path_results = Path("results/mandelbrot")
csv_paths = [path_results / f for f in os.listdir(path_results) if f.endswith(".csv")]
dataframes = [get_dataframe(p) for p in sorted(csv_paths)]
dataframe = pd.concat(dataframes)
df = dataframe

In [ ]:
# Ensure required columns are present
required = {"language","src_filename","size","runtime"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# Parse language into family + opt level, without losing raw rows
def split_lang(s: str):
    m = re.match(r"^\s*([A-Za-z\+\-]+(?:\s*[A-Za-z\-]+)?)\s*(?:(-O[0-3]|-Ofast))?\s*$", str(s))
    if not m:
        return s.strip(), "unspecified"
    fam = m.group(1).strip()
    opt = (m.group(2) or "unspecified").strip()
    return fam, opt

fam_opt = df["language"].apply(split_lang)
df["lang_family"] = fam_opt.apply(lambda x: x[0])
df["opt_level"]   = fam_opt.apply(lambda x: x[1])

# Orders for nice, consistent axes
opt_order  = ["-O1", "-O2", "-O3",]
size_order = sorted(df["size"].dropna().unique())
lang_order = sorted(df["lang_family"].dropna().unique(), key=lambda s: ("mojo" not in s.lower(), s.lower()))

# Cast to categoricals for stable plotting order
df["opt_level"]   = pd.Categorical(df["opt_level"],   categories=opt_order, ordered=True)
df["size"]        = pd.Categorical(df["size"],        categories=size_order, ordered=True)
df["lang_family"] = pd.Categorical(df["lang_family"], categories=lang_order, ordered=False)

df = df[df["opt_level"] != "unspecified"]
df = df[df["opt_level"] != "-Ofast"]

In [ ]:
g = sns.catplot(
    data=df,
    x="opt_level", y="runtime",
    hue="lang_family",
    row="src_filename", col="size",
    order=opt_order, hue_order=lang_order,
    kind="bar",
    errorbar=("pi", 95),  # 95% percentile interval
    capsize=0.15,
    sharey=False,
    height=3.8, aspect=1.2
)

g.set_axis_labels("Optimization level", "Runtime (units)")
g.set_titles("{row_name} — size {col_name}")
g.add_legend(title="Language")

#plt.tight_layout()
plt.show()

In [ ]:
export_fig(g, fig_dir / "mandelbrot.pdf")